In [1]:
! nvidia-smi

Tue Nov 09 02:26:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 456.71       Driver Version: 456.71       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208... WDDM  | 00000000:0D:00.0  On |                  N/A |
|  0%   35C    P8    27W / 260W |    798MiB / 11264MiB |     31%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [2]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn.functional as F
from transformers import BertForSequenceClassification, AutoTokenizer#,BertTokenizer, AutoConfig
# from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, EvalPrediction
from torch.utils.data import TensorDataset, DataLoader


from transformers import AdamW
import os

import csv

from tqdm import tqdm

In [3]:


train_dataset = []
with open('ratings_train.csv', 'r', encoding='utf-8') as fd:
    reader = csv.reader(fd)
    for idx, (_, sentence, rating) in enumerate(reader):
        if idx == 0:
            continue
        train_dataset.append((sentence, int(rating)))        # 0 for positive, 1 for negative
print(len(train_dataset))

150000


In [4]:
# import csv

test_dataset = []
with open('ratings_test.csv', 'r', encoding='utf-8') as fd:
    reader = csv.reader(fd)
    for idx, (sid, sentence) in enumerate(reader):
        if idx == 0:
            continue
        test_dataset.append((sid, sentence))        # 0 for positive, 1 for negative
print(len(test_dataset))

sids, test_sentences = zip(*test_dataset)

50000


In [5]:
device = torch.device("cuda")

# config = AutoConfig.from_pretrained('bert-base-multilingual-cased', num_labels=2)
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
# model = AutoModelForSequenceClassification.from_pretrained('bert-base-multilingual-cased', config=config)
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [6]:
data = {'id': [],
        'sentence': [],
        'label': []}

# 파일 불러오기
with open("ratings_train.csv", 'r', encoding='utf-8') as f:
    train = csv.reader(f)
    for i, example in enumerate(train):
        if i == 0:
            continue
        data['id'].append(example[0])
        data['sentence'].append(example[1])
        data['label'].append(int(example[2]))

In [7]:
train_df = pd.read_csv('ratings_train.csv') #train_df = pd.read_csv('ratings_train.csv', sep='\t')
test_df = pd.read_csv('ratings_test.csv')
train_df.dropna(inplace=True)
test_df.fillna('', inplace=True)
# list(train_df.iloc[:, 1])
tokenized_train_data = tokenizer(list(train_df.iloc[:, 1]), padding = True,  return_tensors='pt', add_special_tokens=True)
all_input_ids = tokenized_train_data.data['input_ids']
all_token_type_ids = tokenized_train_data.data['token_type_ids']
attention_mask = tokenized_train_data.data['attention_mask']
label = torch.tensor(train_df.iloc[:, 2],dtype=torch.long)

train_dataset = TensorDataset(all_input_ids, all_token_type_ids, attention_mask, label)



In [8]:
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [9]:
data = {'id': [],
        'sentence': [],
        'label': []}

# 파일 불러오기
with open("ratings_train.csv", 'r', encoding='utf-8') as f:
    train = csv.reader(f)
    for i, example in enumerate(train):
        if i == 0:
            continue
        data['id'].append(example[0])
        data['sentence'].append(example[1])
        data['label'].append(int(example[2]))

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-6)

epochs = 10

model.train()
model.cuda()

for epoch in range(epochs):
    losses = []
    for batch_idx, batch in tqdm(enumerate(train_loader),total=len(train_loader)):
        batch = tuple(t.cuda() for t in batch)
        inputs = {'input_ids': batch[0],
            'token_type_ids': batch[1],
            'attention_mask': batch[2],
            'labels': batch[3]}


        optimizer.zero_grad()

        output = model(**inputs)

        loss = output.loss

        loss.backward()
        optimizer.step()

        losses.append(np.array(loss.cpu().detach()))
        if batch_idx % 100 == 0:
            print(batch_idx, "Current loss:", np.mean(losses))
            losses = []

        # if itr % p_itr == 0:
        #     print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            
        #     # if total_correct/total_len > best_accuracy:
        #     #     best_accuracy = total_correct/total_len
        #     #     torch.save(model.state_dict(),os.path.join('./','bert_best.pth'))
                
        #     total_loss = 0
        #     total_len = 0
        #     total_correct = 0

        # itr+=1

In [ ]:
# Evaluate
data = {'id': [],
        'sentence': [],
        'label': []}

# 파일 불러오기
with open("ratings_test.csv", 'r', encoding='utf-8') as f:
    train = csv.reader(f)
    for i, example in enumerate(train):
        if i == 0:
            continue
        data['id'].append(example[0])
        data['sentence'].append(example[1])

In [ ]:
tokenized_output = tokenizer(data['sentence'], padding=True, return_tensors='pt')

all_input_ids = tokenized_output.data['input_ids']
all_token_type_ids = tokenized_output.data['token_type_ids']
attention_mask = tokenized_output.data['attention_mask']

test_dataset = TensorDataset(all_input_ids, all_token_type_ids, attention_mask)

BATCH_SIZE = 32
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
predictions = []

with torch.no_grad():
    for batch_idx, batch in tqdm(enumerate(test_loader), total=len(test_loader)):
            batch = tuple(t.cuda() for t in batch)
            inputs = {'input_ids': batch[0],
                      'token_type_ids': batch[1],
                      'attention_mask': batch[2]}

            output = model(**inputs)
            
            logits = output[0]
            predictions += torch.argmax(logits, dim=1)

In [ ]:
predictions = pd.DataFrame(predictions)
predictions.to_csv('sampleSubmission.csv', index=False)